In [1]:
! pip install skorch

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
     fbeta_score,
     confusion_matrix,
     roc_auc_score,
     roc_curve,
     mean_squared_error,
     precision_recall_curve,
     PrecisionRecallDisplay,
     recall_score,
     precision_score
)
from scipy import stats as st
from math import ceil

from sklearn.linear_model import LinearRegression
from catboost import *

from time import time

from sklearn.model_selection import (
    GridSearchCV,
    KFold,
    RandomizedSearchCV,
    cross_val_score
)
import optuna
from tqdm import tqdm

import torch
import torch.nn as nn
torch.cuda.is_available()

from skorch import *
from skorch.callbacks import EpochScoring, EarlyStopping

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
RANDOM_STATE = 1220

In [3]:
df = pd.read_csv('./taxi.csv', index_col=[0], parse_dates=[0])

In [4]:
df.sort_index(inplace=True)


In [5]:
df = df.resample('1H').sum()

In [6]:
df

,num_orders
datetime,
2018-03-01 00:00:00,124
2018-03-01 01:00:00,85
2018-03-01 02:00:00,71
2018-03-01 03:00:00,66
2018-03-01 04:00:00,43
...,...
2018-08-31 19:00:00,136
2018-08-31 20:00:00,154
2018-08-31 21:00:00,159


In [7]:
def make_features(data, max_lag, rolling_mean_size):
    data['year'] = data.index.year
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek
    
    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['num_orders'].shift(lag)

    data['rolling_mean'] = data['num_orders'].shift(1).rolling(rolling_mean_size).mean()

In [8]:
make_features(df, 100, 200)

train, test = train_test_split(df, shuffle=False, test_size=0.1)
train = train.dropna()



/var/folders/n5/30cd9v_56sz_qrnt8mp_9df40000gn/T/ipykernel_5274/705408069.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['lag_{}'.format(lag)] = data['num_orders'].shift(lag)
/var/folders/n5/30cd9v_56sz_qrnt8mp_9df40000gn/T/ipykernel_5274/705408069.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['lag_{}'.format(lag)] = data['num_orders'].shift(lag)
/var/folders/n5/30cd9v_56sz_qrnt8mp_9df40000gn/T/ipykernel_5274/705408069.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of c

In [9]:
def info_df(df):
    print('------------------------------')
    print('| Информация о наборе данных |')
    print('------------------------------')
    df.info()
    print('-----------------------------------------')
    print('| Первые и последние 5 строчек датасета |')
    print('-----------------------------------------')
    display(df)
    print('--------------------')
    print('| Сумма дубликатов |')
    print('--------------------')
    print(df.duplicated().sum())
    for i in df.select_dtypes(include='object').columns.to_list():
        print('--------------------------------------')
        print(f'| Уникальные значения признака {i} |')
        print('--------------------------------------')
        print(df[i].unique())

In [10]:
info_df(train)

------------------------------
| Информация о наборе данных |
------------------------------
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3774 entries, 2018-03-09 08:00:00 to 2018-08-13 13:00:00
Freq: H
Columns: 106 entries, num_orders to rolling_mean
dtypes: float64(101), int64(5)
memory usage: 3.1 MB
-----------------------------------------
| Первые и последние 5 строчек датасета |
-----------------------------------------


,num_orders,year,month,day,dayofweek,lag_1,lag_2,lag_3,lag_4,lag_5,...,lag_92,lag_93,lag_94,lag_95,lag_96,lag_97,lag_98,lag_99,lag_100,rolling_mean
datetime,,,,,,,,,,,,,,,,,,,,,
2018-03-09 08:00:00,36,2018,3,9,4,16.0,4.0,1.0,30.0,31.0,...,85.0,62.0,50.0,59.0,31.0,14.0,3.0,16.0,34.0,55.150
2018-03-09 09:00:00,43,2018,3,9,4,36.0,16.0,4.0,1.0,30.0,...,37.0,85.0,62.0,50.0,59.0,31.0,14.0,3.0,16.0,54.710
2018-03-09 10:00:00,43,2018,3,9,4,43.0,36.0,16.0,4.0,1.0,...,58.0,37.0,85.0,62.0,50.0,59.0,31.0,14.0,3.0,54.500
2018-03-09 11:00:00,49,2018,3,9,4,43.0,43.0,36.0,16.0,4.0,...,45.0,58.0,37.0,85.0,62.0,50.0,59.0,31.0,14.0,54.360
2018-03-09 12:00:00,24,2018,3,9,4,49.0,43.0,43.0,36.0,16.0,...,59.0,45.0,58.0,37.0,85.0,62.0,50.0,59.0,31.0,54.275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-13 09:00:00,137,2018,8,13,0,91.0,39.0,66.0,83.0,143.0,...,79.0,123.0,85.0,149.0,139.0,78.0,28.0,34.0,63.0,111.980
2018-08-13 10:00:00,156,2018,8,13,0,137.0,91.0,39.0,66.0,83.0,...,65.0,79.0,123.0,85.0,149.0,139.0,78.0,28.0,34.0,112.095
2018-08-13 11:00:00,144,2018,8,13,0,156.0,137.0,91.0,39.0,66.0,...,80.0,65.0,79.0,123.0,85.0,149.0,139.0,78.0,28.0,112.140


--------------------
| Сумма дубликатов |
--------------------
0


In [11]:
features_train = train.drop('num_orders', axis=1)
target_train = train['num_orders']

features_test = test.drop('num_orders', axis=1)
target_test = test['num_orders']

In [12]:
vif_data = pd.DataFrame()
vif_data["feature"] = features_train.columns

# вычисление VIF для каждого признака
vif_data["VIF"] = [variance_inflation_factor(features_train.values, i) \
                          for i in range(len(features_train.columns))]

print(vif_data)

          feature        VIF
0            year  32.290737
1           month  24.153539
2             day   2.680706
3       dayofweek   1.400267
4           lag_1   3.072847
..            ...        ...
100        lag_97   3.127772
101        lag_98   3.125845
102        lag_99   3.116723
103       lag_100   3.085710
104  rolling_mean  63.211573

[105 rows x 2 columns]


In [13]:
VIF_features_drop = vif_data.query('VIF > 9.0')['feature']
VIF_features_drop

0              year
1             month
104    rolling_mean
Name: feature, dtype: object

In [15]:
features_train_vif = features_train.drop(VIF_features_drop.values, axis=1)
features_test_vif = features_test.drop(VIF_features_drop.values, axis=1)

In [16]:
X_train, X_test, Y_train, Y_test = torch.tensor(features_train_vif.values, dtype=torch.float32),\
                                   torch.tensor(features_test_vif.values, dtype=torch.float32),\
                                   torch.tensor(target_train.values, dtype=torch.float32),\
                                   torch.tensor(target_test.values, dtype=torch.float32)

In [17]:
scaler = StandardScaler()
X_train_slr = torch.tensor(scaler.fit_transform(X_train), dtype=torch.float32)
X_test_slr = torch.tensor(scaler.transform(X_test), dtype=torch.float32)

In [18]:
def metrics(target, prediction, prediction_proba):
    print("F-beta:",fbeta_score(target,prediction,average='macro',beta=2))
    print("AUC-ROC:", roc_auc_score(target, prediction_proba[:, 1]))
    
    fpr, tpr, thresholds = roc_curve(target, prediction_proba[:, 1])
    cm_matrix = pd.DataFrame(data=confusion_matrix(target, prediction), 
                                columns=['Actual Positive:1', 'Actual Negative:0'], 
                                index=['Predict Positive:1', 'Predict Negative:0'])
    tp = cm_matrix['Actual Positive:1']['Predict Positive:1']
    fp = cm_matrix['Actual Positive:1']['Predict Negative:0']
    fn = cm_matrix['Actual Negative:0']['Predict Positive:1']
    tn = cm_matrix['Actual Negative:0']['Predict Negative:0']
    print('Precision =', round(tp / (tp + fp), 3))
    print('Recall = ', round(tp / (tp + fn), 3))

    fig, axes = plt.subplots(ncols=2, figsize=(15, 5))

    sns.heatmap(cm_matrix, annot=True, fmt='d', 
                cmap=sns.diverging_palette(220, 10, as_cmap=True), ax=axes[0])
    axes[1].plot([0, 1], [0, 1], linestyle='--')
    axes[1].plot(fpr, tpr)


    axes[0].title.set_text('Матрица ошибок')
    axes[1].title.set_text('ROC-кривая')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.tight_layout()
    plt.show()

In [19]:
class NeuralNet(nn.Module):
    def __init__(self, input_dim: int, hidden_lyrs: list[int], output_dim: int, batch_norm=False, dropout=False):
        super(NeuralNet, self).__init__()
        self.relu = nn.ReLU()
        self.batch_norm = batch_norm
        self.dropout = dropout
        self.hidden_lyrs = hidden_lyrs
        if batch_norm == False and dropout == False:
            if len(hidden_lyrs) > 1:
                self.syn_in = nn.Linear(input_dim, hidden_lyrs[0])
                self.hidden_syns = [nn.Linear(hidden_lyrs[i], hidden_lyrs[i+1]) for i in range(0, len(hidden_lyrs)-1)]
                self.syn_out = nn.Linear(hidden_lyrs[-1], output_dim)
            else:
                self.syn_in = nn.Linear(input_dim, hidden_lyrs[0])
                self.hidden_syns = []
                self.syn_out = nn.Linear(hidden_lyrs[0], output_dim)
        elif batch_norm == True and dropout == False:
            if len(hidden_lyrs) > 1:
                self.syn_in = nn.Linear(input_dim, hidden_lyrs[0])
                self.hidden_syns = [[nn.Linear(hidden_lyrs[i], hidden_lyrs[i+1]), nn.BatchNorm1d(hidden_lyrs[i+1])] for i in range(len(hidden_lyrs)-1)]
                self.syn_out = nn.Linear(hidden_lyrs[-1], output_dim)
            else:
                self.syn_in = nn.Linear(input_dim, hidden_lyrs[0])
                self.hidden_syns = []
                self.syn_out = nn.Linear(hidden_lyrs[0], output_dim)
        elif batch_norm == False and dropout == True:
            if len(hidden_lyrs) > 1:
                self.syn_in = nn.Linear(input_dim, hidden_lyrs[0])
                self.hidden_syns = [[nn.Linear(hidden_lyrs[i], hidden_lyrs[i+1]), nn.Dropout(np.random.uniform(0.1, 0.5))] for i in range(len(hidden_lyrs)-1)]
                self.syn_out = nn.Linear(hidden_lyrs[-1], output_dim)
            else:
                self.syn_in = nn.Linear(input_dim, hidden_lyrs[0])
                self.hidden_syns = []
                self.syn_out = nn.Linear(hidden_lyrs[0], output_dim)
        else:
            if len(hidden_lyrs) > 1:
                self.syn_in = nn.Linear(input_dim, hidden_lyrs[0])
                self.hidden_syns = [[nn.Linear(hidden_lyrs[i], hidden_lyrs[i+1]), nn.BatchNorm1d(hidden_lyrs[i+1]), nn.Dropout(np.random.uniform(0.1, 0.5))] for i in range(len(hidden_lyrs)-1)]
                self.syn_out = nn.Linear(hidden_lyrs[-1], output_dim)
            else:
                self.syn_in = nn.Linear(input_dim, hidden_lyrs[0])
                self.hidden_syns = []
                self.syn_out = nn.Linear(hidden_lyrs[0], output_dim)
        


    def forward(self, x):
        if self.batch_norm == False and self.dropout == False:
            out = self.relu(self.syn_in(x))
            if len(self.hidden_syns) > 1: 
                for syn in self.hidden_syns:
                    out = self.relu(syn(out))
            return self.relu(self.syn_out(out))
        elif self.batch_norm == True and self.dropout == False:
            f_b = nn.BatchNorm1d(self.hidden_lyrs[0])
            out = self.relu(f_b(self.syn_in(x)))
            if len(self.hidden_syns) > 1:
                for syn in self.hidden_syns:
                    out = self.relu(syn[1](syn[0](out)))
            return self.relu(self.syn_out(out))
        elif self.batch_norm == False and self.dropout == True:
            f_d = nn.Dropout(np.random.uniform(0.1, 0.5))
            out = f_d(self.relu(self.syn_in(x)))
            if len(self.hidden_syns) > 1:
                for syn in self.hidden_syns:
                    out = syn[1](self.relu(syn[0](out)))
            return self.relu(self.syn_out(out))
        else:
            f_d = nn.Dropout(np.random.uniform(0.1, 0.5))
            f_b = nn.BatchNorm1d(self.hidden_lyrs[0])
            out = f_d(self.relu(f_b(self.syn_in(x))))
            if len(self.hidden_syns) > 1:
                for syn in self.hidden_syns:
                    out = syn[2](self.relu(syn[1](syn[0](out))))
            return self.relu(self.syn_out(out))

In [20]:
# Функция train_model предназначена для обучения модели и принимает в качестве параметров:
#   model       ->  Объект класса NeuralNet
#   batch_size  ->  Размер батча
#   optimizer   ->  Функция оптимизации
#   X,Y         ->  Тензор признаков и целевой соответсвтенно
#   epochs      ->  Количество эпох
#   btnr, drpt  ->  BatchNormaliztion и Dropout соответсвтенно
#
# Функцией потерь является MSE, но в лог выводится RMSE
def train_model(model: NeuralNet, batch_size: int, optimizer, 
                X: torch.Tensor, Y: torch.Tensor, epochs: int, btnr=False, drpt=False):
    num_batches = ceil(len(X)/batch_size)
    criterion = nn.MSELoss()
    accumulation_iteration = 5
    for i in range(epochs):
        order = np.random.permutation(len(X))
        optimizer.zero_grad()
        for batch_idx in range(num_batches):
            start_index = batch_idx * batch_size
            

            batch_indexes = order[start_index:start_index+batch_size]
            X_batch = X[batch_indexes]
            y_batch = Y[batch_indexes]

            preds = model(X_batch).flatten()

            loss = criterion(preds.ravel(), y_batch) / accumulation_iteration

            loss.backward()
            optimizer.step()
            if ((batch_idx + 1) % accumulation_iteration == 0) or (batch_idx + 1 == num_batches):
                optimizer.step()
                optimizer.zero_grad()
        
        if i % 100 == 0 or i == epochs-1:
            print("Epoch: {} RMSE : {:.2f}".format(i, torch.sqrt(loss)))

In [21]:
model_lr = LinearRegression(n_jobs=10, copy_X=True)

start_time_lr = time()

model_lr.fit(features_train_vif, target_train)

end_time_lr = time()

lr_model_time_fit = (end_time_lr-start_time_lr) / 60

In [22]:
print('Время обучение модели: {0:.5f} сек.'. format(lr_model_time_fit))

Время обучение модели: 0.00059 сек.


In [23]:
start_time_lr = time()
predicted = model_lr.predict(features_train_vif)
end_time_lr = time()

lr_model_time_predict = (end_time_lr-start_time_lr) / 60

In [24]:
model_lr_cv = LinearRegression(n_jobs=10, copy_X=True)

start_time_lr = time()
folds = KFold(n_splits = 4, shuffle = True, random_state = RANDOM_STATE)
scores = cross_val_score(model_lr_cv, features_train_vif, target_train, scoring='neg_root_mean_squared_error', cv=4)
end_time_lr = time()

lr_model_time_cv = (end_time_lr-start_time_lr) / 60

In [25]:
print("RMSE кросс валидации: {0:.2f}.". \
    format(scores.max() * -1))

RMSE кросс валидации: 18.49.


In [26]:
model_lr.predict(features_train_vif)

array([ 39.11121197,  52.50018944,  41.45665162, ..., 110.36595152,
       115.97329603, 112.47841392])

In [27]:
INPUT = len(features_train_vif.columns)
HIDDEN = [75,50,25,10]
OUTPUT = 1

In [28]:
model_1 = NeuralNet(input_dim=INPUT, hidden_lyrs=HIDDEN, output_dim=OUTPUT)
                    
optimizer = torch.optim.Adam(model_1.parameters(), lr=.01)
train_model(model_1, 100, optimizer,X_train_slr,Y_train,1000)

Epoch: 0 RMSE : 39.46
Epoch: 100 RMSE : 35.48
Epoch: 200 RMSE : 41.73
Epoch: 300 RMSE : 35.84
Epoch: 400 RMSE : 36.83
Epoch: 500 RMSE : 40.78
Epoch: 600 RMSE : 36.01
Epoch: 700 RMSE : 40.98
Epoch: 800 RMSE : 39.80
Epoch: 900 RMSE : 36.96
Epoch: 999 RMSE : 39.03


In [29]:

skorch_regressor = NeuralNetRegressor(module=model_1, 
                                      device=DEVICE,  
                                      verbose=3,
                                      batch_size=100,
                                      optimizer=torch.optim.Adam, 
                                      max_epochs=2500,
                                      lr=0.01
                                      train_split=4,
                                      criterion =nn.MSELoss,
                                      callbacks=[
                ('val_rmse', EpochScoring(scoring="neg_root_mean_squared_error", lower_is_better=True, name='RMSE')),
                ('estoper', EarlyStopping( lower_is_better=True, monitor='RMSE')),
            ],
                                     )

In [39]:
 
params = {
    
    'batch_norm': [True, False],
    'dropout': [True, False],
    'lr': [1e-2, 1e-3, 1e-4]
}

In [40]:
grid = RandomizedSearchCV(skorch_regressor, params, cv=3, scoring="neg_root_mean_squared_error", verbose=3) 
grid.fit(X_train_slr, Y_train.reshape(-1,1)) 

Fitting 3 folds for each of 10 candidates, totalling 30 fits


ValueError: Invalid parameter 'dropout' for estimator <class 'skorch.regressor.NeuralNetRegressor'>[uninitialized](
  module=NeuralNet(
    (relu): ReLU()
    (syn_in): Linear(in_features=102, out_features=75, bias=True)
    (syn_out): Linear(in_features=10, out_features=1, bias=True)
  ),
). Valid parameters are: ['module', 'criterion', 'optimizer', 'lr', 'max_epochs', 'batch_size', 'iterator_train', 'iterator_valid', 'dataset', 'train_split', 'callbacks', 'predict_nonlinearity', 'warm_start', 'verbose', 'device', 'compile', 'use_caching', '_params_to_validate'].